In [1]:
import requests
import json, ndjson
from collections import Counter

with open("secret.csv", "r") as r:
    data = [line.replace("\n","").split(",") for line in r.readlines()]
    username = data[0][1]
    token = data[1][1]

In [2]:
url = "https://lichess.org"
print(f"Gathering data for {username}")
response = requests.get(
  f'https://lichess.org/api/games/user/{username}',
  params={
    "perfType" : "blitz",
    "rated" :  "true",
    "opening" : "true",
    "moves" : "false"
  },
  headers={
    'Authorization': f'Bearer {token}', # Need this or you will get a 401: Not Authorized response
    "Accept": "application/x-ndjson"
  }
)

Gathering data for sirsnakerb


In [3]:
# Parse application/x-ndjson into list of JSON objects
games = []
ndjson = response.content.decode().split('\n')

for json_obj in ndjson:
    if json_obj:
        games.append(json.loads(json_obj))

In [4]:
len(games)

3685

In [5]:
games[1]

{'id': '8CUOWCNu',
 'rated': True,
 'variant': 'standard',
 'speed': 'blitz',
 'perf': 'blitz',
 'createdAt': 1603039771257,
 'lastMoveAt': 1603040105442,
 'status': 'mate',
 'players': {'white': {'user': {'name': 'sirsnakerb', 'id': 'sirsnakerb'},
   'rating': 1412,
   'ratingDiff': 5},
  'black': {'user': {'name': 'ITHM', 'id': 'ithm'},
   'rating': 1338,
   'ratingDiff': -4}},
 'winner': 'white',
 'opening': {'eco': 'C00',
  'name': 'French Defense: La Bourdonnais Variation',
  'ply': 3},
 'clock': {'initial': 300, 'increment': 0, 'totalTime': 300}}

In [13]:
# seperate games for black and white games
white_games, black_games = [], []
for game in games:
    if game.get("opening"):
        try:
            if game.get("players").get("white").get("user").get("name") == username:
                white_games.append(game)
            else:
                black_games.append(game)
        except:
            print(game)

# game["opening"]["name"]

{'id': '35WNZz7m', 'rated': True, 'variant': 'standard', 'speed': 'blitz', 'perf': 'blitz', 'createdAt': 1603040491280, 'lastMoveAt': 1603040849570, 'status': 'timeout', 'players': {'white': {'user': {'name': 'sirsnakerb', 'id': 'sirsnakerb'}, 'rating': 1417, 'ratingDiff': 6}, 'black': {'user': {'name': 'therudegoat', 'id': 'therudegoat'}, 'rating': 1435, 'ratingDiff': -6}}, 'winner': 'white', 'opening': {'eco': 'B12', 'name': 'Caro-Kann Defense: Advance Variation', 'ply': 5}, 'clock': {'initial': 300, 'increment': 0, 'totalTime': 300}}
{'id': '8CUOWCNu', 'rated': True, 'variant': 'standard', 'speed': 'blitz', 'perf': 'blitz', 'createdAt': 1603039771257, 'lastMoveAt': 1603040105442, 'status': 'mate', 'players': {'white': {'user': {'name': 'sirsnakerb', 'id': 'sirsnakerb'}, 'rating': 1412, 'ratingDiff': 5}, 'black': {'user': {'name': 'ITHM', 'id': 'ithm'}, 'rating': 1338, 'ratingDiff': -4}}, 'winner': 'white', 'opening': {'eco': 'C00', 'name': 'French Defense: La Bourdonnais Variation',

In [144]:
most_common = Counter(openings).most_common()
most_common_percentage = [(opening[0], round(opening[1]/len(openings)*100,2)) for opening in most_common]

In [145]:
most_common_percentage

[("Queen's Pawn Game: Mason Variation", 20.45),
 ("King's Pawn Game: Busch-Gass Gambit", 6.14),
 ('Horwitz Defense', 4.77),
 ('Englund Gambit', 3.64),
 ("King's Pawn Game: Wayward Queen Attack", 2.73),
 ('Four Knights Game: Italian Variation', 2.73),
 ('Mikenas Defense', 2.5),
 ('Indian Game', 2.5),
 ("Queen's Pawn Game", 2.5),
 ("Queen's Pawn", 2.27),
 ("King's Pawn Game: Leonardis Variation", 2.05),
 ("Van't Kruijs Opening", 1.59),
 ('Caro-Kann Defense', 1.59),
 ("King's Pawn Game: Damiano Defense", 1.36),
 ("Queen's Pawn Game: Chigorin Variation", 1.36),
 ('Italian Game', 1.14),
 ('Nimzowitsch Defense', 1.14),
 ('Modern Defense', 1.14),
 ("Bishop's Opening", 1.14),
 ("Queen's Pawn Game: Veresov Attack, Alburt Defense", 1.14),
 ('Philidor Defense', 1.14),
 ("King's Pawn Game", 1.14),
 ('Four Knights Game', 0.91),
 ("Queen's Gambit Declined: Baltic Defense", 0.91),
 ("Bishop's Opening: Boi Variation", 0.91),
 ('Englund Gambit Declined: Reversed French', 0.68),
 ('Englund Gambit Declin